# Lecture 4: Data Quality & Cleaning Essentials - Professional Data Preparation

## Learning Objectives

By the end of this lecture, you will be able to:
- Identify and categorize different types of data quality issues in transportation datasets
- Apply systematic approaches to detect inconsistencies, missing data, and outliers
- Choose appropriate strategies for handling data quality issues
- Implement professional data cleaning workflows using pandas methods

---

## 1. The Reality of Real-World Transportation Data

As your consulting work progresses, you'll quickly discover a fundamental truth: **real-world data is messy**. The clean datasets you see in textbooks don't exist in professional practice. Your bike-sharing client's data comes from sensors that malfunction, weather stations that go offline, and databases that occasionally corrupt records.

This messiness isn't just a technical inconvenience - it's a business-critical challenge. **Poor data quality can lead to incorrect demand predictions**, resulting in empty bike stations when customers need them or overflow situations where returning bikes becomes impossible. These operational failures directly impact customer satisfaction and revenue.

Your role as a professional consultant is to **transform messy, incomplete data into reliable foundations for business decision-making**. This requires systematic approaches, clear documentation, and transparent communication about data limitations and cleaning procedures.

## 2. Common Data Quality Issues in Transportation Systems

Messiness isn’t a single problem—it comes in many forms, each affecting your analysis in different ways. As a consultant, your job is to **recognize these issues early** and decide how to handle them before they undermine your predictions.

### 2.1. Understanding Data Quality Dimensions

Data quality exists along multiple dimensions, each of which can influence your analysis in different ways. Let’s break down the five most important ones and see how they appear in transportation data:

1. **Completeness**
   - *Definition:* Whether all expected values are present in the dataset.
   - *Why it matters:* Missing values create blind spots, especially when the missingness is not random.
   - *Example:* If bike-sharing sensors fail during storms, you may lose exactly the data needed to understand weather impacts on demand.

2. **Accuracy**
   - *Definition:* The extent to which recorded values reflect reality.
   - *Why it matters:* Inaccurate values can mislead both descriptive analysis and predictive models.
   - *Example:* A temperature of -50°C recorded in Washington D.C. in July is a clear sensor error that could confuse demand models.

3. **Consistency**
   - *Definition:* Whether data follows uniform formats, units, and scales.
   - *Why it matters:* Inconsistent formats can corrupt calculations and comparisons.
   - *Example:* Mixing Celsius and Fahrenheit in the same column, or having timestamps in multiple formats, leads to corrupted analysis.

4. **Validity**
   - *Definition:* Whether values fall within logical or physically possible ranges.
   - *Why it matters:* Invalid data points indicate measurement or collection errors.
   - *Example:* Negative bike counts or humidity above 100% are impossible values that reveal collection problems.

5. **Uniqueness**
   - *Definition:* Whether each observation is recorded only once.
   - *Why it matters:* Duplicate records inflate usage counts and distort demand predictions.
   - *Example:* If the same rental transaction is logged twice, it looks like demand is higher than it really was.

Together, these five dimensions reveal how data quality problems can undermine predictions in different ways. Recognizing these distinct aspects helps you target specific problems rather than applying generic "data cleaning" approaches.

### 2.2. Missing Data Patterns and Business Implications

Not all missing data is created equal. In transportation systems, gaps in the dataset often occur under very specific conditions—the very conditions you want to analyze. Understanding when and why data goes missing helps you choose the right handling strategies and communicate risks to clients.

**Weather-Driven Data Loss**

- The most common pattern involves sensor failures during extreme weather events. Storms, heavy rain, and temperature extremes can knock out monitoring equipment precisely when weather has the strongest influence on bike usage. This creates a double problem: you lose data exactly when you need it most, and standard statistical assumptions about "random" missingness don't apply.

**Operational and Maintenance Gaps**

- Planned maintenance creates predictable but significant gaps in transportation data. These periods often overlap with major infrastructure changes—like opening new stations or updating software systems—meaning that ignoring the operational context could hide important business insights about system growth and performance.

**Network Effects and Cascade Failures**

- Transportation systems are interconnected. When a high-demand station goes offline, neighboring stations typically experience unusual demand spikes. Without data from the offline station, it becomes difficult to distinguish between genuine demand growth and temporary displacement of riders to nearby locations.

**Peak Period Vulnerabilities**

- Finally, data failures during rush hours pose special challenges because peak-period predictions drive critical business decisions. Missing data during these high-stakes periods requires more sophisticated handling strategies than gaps occurring during quiet off-peak hours when the business impact is minimal.

These four patterns demonstrate why missing data analysis goes beyond simple counts and percentages. Each pattern creates specific risks for demand forecasting and requires tailored handling strategies. By understanding when and why data disappears, you can make informed decisions about imputation, communicate limitations clearly to clients, and avoid building models on unreliable foundations.

## 3. Data Quality Assessment Process

As a consultant, clients will expect you to follow a systematic, defensible process for data quality assessment — not ad-hoc checking. The 4-step process we'll use gives you a professional framework that you can explain and justify to any client:

1. **Quick Data Quality Checks** – your first diagnostic scan to flag obvious issues.
2. **Time Series Integrity Check** – systematic analysis of temporal continuity.
3. **Outlier Detection** – comprehensive detection of anomalies.
4. **Missing Data Detection** – detailed analysis of completeness.

This systematic approach demonstrates expertise and builds client confidence from day one. Steps 1, 3, and 4 apply universally across domains. Step 2 becomes critical for time-series data, like the one we usually see in transportation problems.

### 3.1. Quick Data Quality Checks

Quick data quality checks are rapid diagnostic scans that assess whether a dataset is fundamentally sound before investing time in detailed analysis or modeling.

Think of this like a consultant's "triage" — in just a few minutes, you want to know whether the dataset looks broadly reliable, where the biggest risks lie, and which areas deserve closer investigation. When you receive a new dataset from a client, this scan is your first step, not modeling.

These lightweight checks flag obvious issues across structure, value ranges, and cross-variable plausibility. We won't yet explain problems in depth or attempt fixes — that comes later. The goal is rapid risk assessment.

Our quick quality assessment will be based on three steps:

1. Structural Snapshot
2. Value Sanity Checks
3. Cross-Variable Plausibility

**1. Structural Snapshot**

The first thing we do is take a **structural snapshot** of the dataset: how many rows and columns it has, and whether the variables are of the expected type. This step sounds simple, but it’s one of the fastest ways to detect import errors, unexpected row counts, or inconsistencies in data types — all of which can indicate bigger problems lurking beneath the surface.

In [ ]:
import pandas as pd

# Load the Washington D.C. bike-sharing dataset (intentionally messy version)
df = pd.read_csv("https://raw.githubusercontent.com/pmarcelino/predictive-modeling/main/datasets/dataset-teaching-lec-04.csv")

# Check dataset dimensions and data types
print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")
print(df.info())

> **Note:** You may have noticed something unusual here: the results don't look exactly like what we saw in the previous lecture. That's intentional. From this point forward, we'll sometimes work with a slightly modified version of the Washington D.C. dataset. We've made it "messy" on purpose so you can practice handling real-world problems. As a consultant, you'll rarely get a dataset that's perfectly clean — each phase of the course will bring new challenges for you to detect and resolve.

When we run this quick check, a few important concerns stand out immediately:

* The **`holiday` column has missing values**. This means that not every day is properly labeled as a holiday or not — a detail that could easily distort demand forecasts, since holiday patterns differ sharply from regular weekdays.
* The **weather-related variables** such as `temp`, `humidity`, and `windspeed` also contain gaps. Because these are some of the most important explanatory variables in our forecasting model, missingness here reduces our ability to explain variation in bike rentals.
* Most critically, the **`count` column — our target variable — is missing in several rows**. This is a red flag: every missing entry in `count` means lost training data, and the reliability of our model hinges on how much usable demand history we have.

This single, simple scan already tells us that the dataset cannot be used “as is” for modeling. More importantly, it shows why **structural checks are powerful**: with just one command, we’ve uncovered problems in both our explanatory variables and our target.

This is exactly the kind of insight to highlight at the project start: *"Before we can move into forecasting, we've already identified major gaps in the dataset that could affect both explanatory power and prediction accuracy."*

**2. Value Sanity Checks**

After confirming the dataset’s structure, the next step is to ask: *“Do the values themselves make sense?”*

Every variable has **natural boundaries** defined by either business rules or physical limits:

- Bike rentals cannot be negative.
- Humidity must fall between 0% and 100%.
- Local temperatures should stay within climate-appropriate ranges.

Values that fall outside these ranges are not just unusual — they are almost always errors caused by faulty sensors, bad data entry, or processing mistakes.

In [ ]:
# Basic range checks
print("Negative rentals:", (df["count"] < 0).sum())
print("Humidity out of range:", ((df["humidity"] < 0) | (df["humidity"] > 100)).sum())
print("Temperature extremes:", df["temp"].describe()[["min", "max"]])

Running these quick checks reveals three immediate red flags:

- We find **6 cases of negative rentals**, which is logically impossible — you can’t rent fewer than zero bikes. This is most likely a logging or entry error.
- We spot **10 humidity values above 100%**, which is physically impossible. This usually points to a faulty sensor reading or an ingestion problem.
- Finally, the **temperature maximum is close to 100°C**. While summers in Washington D.C. can be hot, they certainly don’t reach boiling point! This extreme value is almost certainly an error that could distort averages or mislead a forecasting model.

Together, these findings show why **range validation is essential**. With a few simple checks, we can identify values that clearly break real-world rules — and if left undetected, they could slip into analysis, biasing results and damaging credibility with clients.

This step builds trust: you demonstrate that you're not just running models blindly, but verifying whether the data itself reflects reality.

**3. Cross-Variable Plausibility**

Numbers can look fine in isolation but make no sense once you compare them across variables. That’s why a good quick check also includes a **plausibility scan across related variables**. In transportation data, the most important relationship to test is usually between **demand** and **context variables** like weather.

For example, common sense (and business experience) tells us that **bike rentals should fall when weather conditions worsen**. If the dataset shows the opposite, that’s a red flag.

In [ ]:
# Quick plausibility check: average rentals by weather condition
avg_rentals_by_weather = df.groupby("weather")["count"].mean()
print(avg_rentals_by_weather)

> **Note:** The weather variable is coded as follows:
> - **1**: Clear, Few clouds, Partly cloudy
> - **2**: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist  
> - **3**: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
> - **4**: Heavy Rain + Ice Pellets + Thunderstorm + Mist, Snow + Fog

At first glance, the results follow expectations:

- Rentals are **highest on clear days (weather = 1)**.
- They gradually decrease as conditions worsen to misty or light rain.

But the final category is suspicious. Under **heavy rain or storms (weather = 4)**, the dataset shows an **average of more than 600 rentals per hour** — even higher than on sunny days.

This makes little business sense: real-world demand should drop sharply in severe weather, not skyrocket. Such an inconsistency usually points to:

- **Mislabelled weather codes**, or
- A **misalignment between weather feeds and rental logs**.

Left uncorrected, such errors could lead to false conclusions like *"bike demand is resilient during storms"* — which in turn could drive poor operational decisions, such as overstocking bikes or overscheduling staff during extreme weather events.

This kind of cross-variable check is a reminder: **some errors only appear when you look at relationships, not just single columns**. That’s why you should test whether the data’s “story” matches real-world logic.

### 3.2. Temporal Continuity Check

Temporal continuity refers to whether your time-series data has a complete, consistent timeline without gaps, duplicates, or misaligned sequences.

Transportation data is inherently time-based. If the timeline itself is broken, then any further cleaning, imputation, or modeling will rest on shaky foundations. Missing hours create false patterns, duplicates skew averages, and misaligned sequences break seasonal analysis.

Before we tackle outliers or missing values, we must verify the timeline's integrity to ensure reliable foundations for all subsequent work. To do so, we run a timeline diagnostic that tells us:

- The first and last timestamp in the dataset.
- How many hours should exist in that range.
- How many unique hours actually exist.
- How many are missing.
- How many duplicate rows we have for the same hour.

This gives us a quick sense of whether the dataset is complete and well-aligned, or whether we’re missing entire blocks of time.

In [ ]:
import pandas as pd

# Ensure datetime is properly parsed
data_path = "https://raw.githubusercontent.com/pmarcelino/predictive-modeling/main/datasets/dataset-teaching-lec-04.csv"
df = pd.read_csv(data_path)
df["datetime"] = pd.to_datetime(df["datetime"], errors="coerce")
df = df.sort_values("datetime").reset_index(drop=True)

# Identify coverage
t_min = df["datetime"].min()
t_max = df["datetime"].max()

# Duplicated timestamps
n_dup_rows = df.duplicated(subset=["datetime"]).sum()

# Build expected hourly range
expected = pd.date_range(t_min, t_max, freq="h")
actual = pd.Index(df["datetime"].unique())

n_expected = len(expected)
n_actual = len(actual)
n_missing_hours = len(expected.difference(actual))

print("=== Timeline quick check ===")
print("time_min:", t_min)
print("time_max:", t_max)
print("expected_hours:", n_expected)
print("present_unique_hours:", n_actual)
print("missing_hours:", n_missing_hours)
print("duplicate_rows:", n_dup_rows)

As we can see, between January 2011 and December 2012, the dataset should contain **17 256 hourly rows**. In reality, it only contains **10 862 unique hours**, leaving **6 394 hours missing**. That's more than a third of the timeline absent — a major structural gap. We also see **24 duplicate rows**, meaning some hours are represented more than once.

This is a critical insight: the raw data cannot be trusted as a continuous timeline. Large gaps undermine seasonal analysis, and duplicates risk double-counting demand. Before any modeling, we need to fix both problems. We will show how to do that in [Chapter 4](#4-data-cleaning-strategies-and-implementation).

### 3.3. Outlier Detection

Outlier detection identifies data points that deviate significantly from the expected pattern — values that are unusually high, unusually low, or inconsistent with the rest of the dataset.

In transportation data, outliers can represent legitimate extreme events (like snowstorms causing demand drops), data collection errors (like negative bike rentals), or operational anomalies (like maintenance affecting normal patterns). Each type requires different treatment strategies.

The goal is not just to detect outliers, but to classify them correctly so that legitimate events are preserved while errors are corrected. This ensures models learn from real patterns rather than data quality issues.

Now that we understand what outliers are, let’s look at the different ways to detect them. Outlier detection methods generally fall into three categories:

1. Statistical Methods
2. Business Logic Methods
3. Temporal Methods

**1. Statistical Methods**

Statistical methods use mathematical formulas to identify unusual values. They don’t require prior knowledge of the transportation system - they just look at how far a data point is from what is “normal” in the dataset.

There are several statistical approaches, such as:

- Z-Score Analysis
- Interquartile Range (IQR) Method
- Modified Z-Score

In this lecture, we will focus on just one example: **Z-Score Analysis**.

A **Z-score** tells us how many “standard steps” (standard deviations) a data point is away from the average.

$$
Z = \frac{x - \mu}{\sigma}
$$

where:

- $x$ = the value we’re checking
- $\mu$ = the mean (average) of the data
- $\sigma$ = the standard deviation (how spread out the data is)

Think of the average bike rentals per day like the “center of gravity” of the data. Most days will be close to that average. The Z-score is like a distance meter: it tells us how far a particular day is from the typical pattern.

- A Z-score of **0** → exactly average.
- A Z-score of **+2** → two steps above average (busier than normal).
- A Z-score of **–3** → three steps below average (quieter than normal).

When a Z-score is bigger than 3 or smaller than –3, the value is far enough from the average that we should pause and ask: *Is this a real event, or is it an error?*

We use Z-scores because they:

- **Standardize values** so we can compare across variables.
- **Give a simple rule of thumb**: beyond 3 = unusual.
- **Provide a quick first filter** before applying more advanced techniques.

Let’s see how this works in practice using the Washington D.C. bike-sharing dataset. We’ll calculate the Z-scores for daily demand (`count`) and flag potential outliers.

In [ ]:
import pandas as pd
import numpy as np

# Load the Washington D.C. bike-sharing dataset
data_path = "https://raw.githubusercontent.com/pmarcelino/predictive-modeling/main/datasets/dataset-teaching-lec-04.csv"
df = pd.read_csv(data_path)

# Convert datetime column to pandas datetime type
df['datetime'] = pd.to_datetime(df['datetime'])

# Aggregate rentals by day
daily_rentals = df.groupby(df['datetime'].dt.date)['count'].sum().reset_index()

# Calculate mean and std
mean = daily_rentals['count'].mean()
std = daily_rentals['count'].std()

# Compute Z-scores
daily_rentals['z_score'] = (daily_rentals['count'] - mean) / std

# Flag outliers (|Z| > 3)
outliers = daily_rentals[daily_rentals['z_score'].abs() > 3]

outliers.head()

Both May 15 and November 3 show exceptionally high rental counts, far beyond typical daily demand. These are unlikely to be ordinary fluctuations. They could represent special city-wide events or anomalies in how trips were logged. As consultants, we need to cross-check these dates with event calendars and system logs to confirm whether these spikes reflect genuine demand or possible data quality issues.

**2. Business Logic Methods**

While statistical methods rely purely on mathematical rules, **business logic methods** use knowledge of the system and its physical constraints to detect outliers. Instead of asking, “Does this number look statistically unusual?”, we ask, “Is this number even possible given how the transportation system works?”

Business logic methods build rules like these based on:

- **Physical constraints**: e.g., a bike station cannot have negative bikes, nor can it rent more bikes than its maximum capacity.
- **Historical ranges**: e.g., demand has never exceeded 1,200 rentals in a day; a value above this threshold is suspicious.
- **Cross-variable checks**: e.g., it shouldn’t be possible to record “heavy rain” alongside “record-high bike usage.”

Let's apply business logic to detect outliers by checking for physical constraint violations in wind speed measurements.

In [ ]:
import pandas as pd

# Load the Washington D.C. bike-sharing dataset
data_path = "https://raw.githubusercontent.com/pmarcelino/predictive-modeling/main/datasets/dataset-teaching-lec-04.csv"
df = pd.read_csv(data_path)

# Convert datetime column to pandas datetime type
df['datetime'] = pd.to_datetime(df['datetime'])

# Check for physical constraint violations in windspeed
invalid_windspeed = df[(df['windspeed'] < 0) | (df['windspeed'] > 60)]

invalid_windspeed.head()

The flagged records show windspeed values of `200.0`, which are far beyond any physically possible measurement for this system. These values clearly indicate sensor or recording errors rather than real-world weather conditions. If left uncorrected, they could distort downstream models, for example by falsely associating extreme winds with normal rental demand.

**3. Temporal Methods**

Transportation data is inherently tied to **time**. Unlike static datasets, values change depending on the hour, day, season, or long-term trends. **Temporal outlier detection methods** focus on identifying unusual data points that break these expected time-based patterns. For example:

- A sudden drop in rentals during a weekday morning rush hour might indicate a system outage.
- A sharp jump in rentals during winter could mean a special event.
- A long-term shift in demand may signal that the system has grown or changed in some way.

Some common temporal approaches include:

- **Change Point Detection**: Identifying sudden structural shifts in the data (e.g., a new station or policy).
- **Seasonal Anomaly Detection**: Checking if values align with expected seasonal patterns.
- **Trend Deviation Analysis**: Comparing current values to long-term growth or decline.

Let's see an example of a seasonal anomaly detection. We'll identify days where demand deviates significantly from seasonal expectations by comparing winter and summer rental patterns, then flagging winter days with unusually high demand and summer days with unusually low demand.

In [ ]:
import pandas as pd

# Load the Washington D.C. bike-sharing dataset
data_path = "https://raw.githubusercontent.com/pmarcelino/predictive-modeling/main/datasets/dataset-teaching-lec-04.csv"
df = pd.read_csv(data_path)

# Convert datetime and build daily_rentals with a month column
df["datetime"] = pd.to_datetime(df["datetime"])
daily_rentals = (
    df.groupby(df["datetime"].dt.date)["count"]
      .sum()
      .reset_index()
      .rename(columns={"datetime": "date"})
)
daily_rentals["date"] = pd.to_datetime(daily_rentals["date"])
daily_rentals["month"] = daily_rentals["date"].dt.month

# Define "winter" as Dec-Feb and "summer" as Jun-Aug
winter_months = [12, 1, 2]
summer_months = [6, 7, 8]

winter_avg = daily_rentals[daily_rentals['month'].isin(winter_months)]['count'].mean()
summer_avg = daily_rentals[daily_rentals['month'].isin(summer_months)]['count'].mean()

# Flag unusual winter days (too close to summer levels)
winter_anomalies = daily_rentals[
    (daily_rentals['month'].isin(winter_months)) & 
    (daily_rentals['count'] > summer_avg * 0.8)
]

# Flag unusual summer days (too close to winter levels)
summer_anomalies = daily_rentals[
    (daily_rentals['month'].isin(summer_months)) & 
    (daily_rentals['count'] < winter_avg * 1.2)
]

seasonal_anomalies = pd.concat([winter_anomalies, summer_anomalies])
seasonal_anomalies.head()

The detected anomalies highlight days in winter months (January and February) with demand levels much closer to what we’d expect in summer. For example, January 7 shows more than double the average rentals for that month. These could indicate unusually warm days that encouraged cycling, or they might reflect special events. In practice, such findings should be validated with weather data or event calendars. This illustrates how seasonal anomaly detection helps identify values that break expected seasonal patterns, providing valuable clues about real-world influences on demand.

### 3.4. Missing Data Detection

Missing data detection is the systematic analysis of gaps in your dataset to understand their patterns, causes, and potential impact on modeling and analysis.

Unlike outliers which are individual problematic values, missing data represents systematic gaps that can undermine entire analyses. Missingness often follows patterns — clustering in certain periods, affecting groups of variables together, or reflecting underlying system failures like sensor outages.

We analyze missing data patterns to design targeted cleaning strategies rather than applying generic fixes. Understanding where, when, and why data goes missing ensures our solutions address root causes and preserve data integrity.

We'll explore missing data through three systematic analyses:

- Quantitative assessment
- Temporal pattern analysis
- Cross-variable analysis

**1. Quantitative Assessment of Missing Data**

A **quantitative assessment** inventories missing values across all columns, showing both you and the client the scale and location of gaps.

In [ ]:
# Count missing values and calculate percentages
missing_counts = df.isnull().sum()
missing_percentages = (missing_counts / len(df)) * 100

# Display side by side for clarity
pd.DataFrame({"Missing": missing_counts, "Percentage": missing_percentages})

Our scan shows three main areas of concern:

- First, the `holiday` column is missing in about **5% of rows**, which means we can't always tell whether a given day was a holiday — a potentially important driver of demand.
- Second, weather-related variables (`temp`, `atemp`, `humidity`, `windspeed`) are missing in around **0.6% of cases each**. That may sound small, but if they go missing together, it likely reflects a sensor or reporting problem.
- Finally, and most importantly, the **target variables** (`count`, `casual`, `registered`) are missing in **146 rows**, or about **1.3% of the dataset**.

Any missing demand values directly reduce the amount of training data available for forecasting, which is bad.

**2. Temporal Pattern Analysis**

In transportation datasets, missing data often clusters in specific periods. This makes **temporal analysis** essential: by grouping missingness across months or seasons, we can check whether the gaps are random or systematically tied to certain time periods.

This matters for forecasting because if missingness is concentrated in peak demand months, any model we build will be biased or incomplete in those periods.

In [ ]:
# Extract year and month into a new column
df['year_month'] = df['datetime'].dt.strftime('%Y-%m')

# Group by the new 'year_month' column and calculate missing values
missing_by_month = df.groupby(df['year_month'])['count'].apply(lambda x: x.isnull().sum())
print(missing_by_month)

The temporal scan reveals that missing demand values are not evenly spread across the dataset. Instead, they cluster heavily in **July 2011 and July 2012**. This points to a systematic issue, such as a recurring sensor outage or reporting gap during summer months. For the client, this has a clear implication: forecasts for peak-season demand may be less reliable unless these gaps are addressed. It’s not just random noise — it’s a structural weakness in the dataset that could distort decision-making during the busiest time of year.

**3. Cross-Variable Analysis of Missing Data**

Sometimes, missingness in one variable aligns with gaps in others. This is an important diagnostic step because it helps distinguish between isolated issues (e.g., a single column not recorded) and **system-wide failures** (e.g., a weather station outage affecting several variables at once).

By checking which variables tend to go missing together, we can form a more realistic hypothesis about the underlying cause.

In [ ]:
# Check rows where at least one weather-related variable is missing
weather_missing = df[df[["temp", "humidity", "windspeed"]].isnull().any(axis=1)]
print(weather_missing.head())

The cross-variable check confirms that missingness is not isolated — entire blocks of weather data (`temp`, `atemp`, `humidity`, `windspeed`) disappear at the same time. For example, on **February 11, 2011**, several consecutive hours show all weather variables missing together. This strongly suggests a **weather station outage or reporting failure**, not random gaps. Recognizing that these variables fail together allows us to design a coordinated cleaning strategy rather than treating each column as an independent problem.

## 4. Data Cleaning Strategies and Implementation

We've diagnosed the problems in our bike-sharing dataset: impossible values, extreme outliers, missing data blocks, and timeline gaps. **Now comes the treatment stage** — this is where clients see the real consulting value.

Diagnosis impresses clients with your analytical rigor, but **cleaning delivers the reliable data foundation** they need for business decisions. While assessment shows what's wrong, cleaning demonstrates how you solve problems systematically and transparently.

This transition from "finding issues" to "fixing issues" represents the shift from diagnostic consultant to solution provider. Clients pay for datasets they can trust, models they can deploy, and insights they can act on.

**Data cleaning** is not cosmetic work to make datasets "look nice." It's a structured process that:

- Distinguishes **errors** from **real events**
- Applies **consistent, rule-based fixes** where possible
- Decides when to **impute or drop** values that cannot be fixed
- Keeps every change **transparent and auditable**

This systematic approach ensures that your cleaning decisions can be explained, defended, and replicated — critical requirements for professional consulting work.

### 4.1. The Unified Cleaning Workflow

Since we are working with time-series data, we will start by **standardizing the timeline** and fixing its structural problems — this is a critical first step that ensures we have a reliable temporal foundation.

From there, we will follow a standard data cleaning workflow that, while always dependent on the specific dataset and business context, can be generalized into a systematic process that learners can apply to their own projects. Once the timeline is reliable, every suspicious value — whether it’s extreme, impossible, or missing — is treated with the same **three-step decision tree**:

1. **Is this an event or an error?**

   - *Event* → keep, but **flag** (e.g., snowstorm, festival).
   - *Error* → continue.

2. **If error: Can I fix it with a rule?**

   - Examples: cap humidity to 100, relabel mis-coded weather categories, set negative rentals to `NaN`.
   - If yes → **fix and flag**.

3. **If cannot fix: Should I impute or drop?**

   - **Predictors (features):** impute if valuable, drop if not.
   - **Target (`count`):** never impute for modeling → drop missing rows.
   - Always **flag** imputations or dropped ranges.

> **Note 1:** **Predictors** are the input variables we use to make predictions (like weather conditions, time of day, or season), while the **target** is the outcome variable we're trying to predict (in our case, bike rental counts).

> **Note 2:** Why flag? We flag because it ensures that every change is **visible, auditable, and explainable**. Flags allow you to:
> 
> - Compare model performance with and without imputed values.
> - Communicate risks to clients (*“July demand is less reliable: 20% of weather values were imputed.”*).
> - Keep a record of what changed and why.

This mindset — *event or error? fix, impute, or drop? always flag* — is the backbone of professional data cleaning.

### 4.2. Standardizing the Timeline

In the previous chapters, we discovered that our timeline is not fully reliable: some hours are missing, while others are duplicated. To move forward confidently, we need to standardize the timeline so that every hour has exactly one valid record.

We will do so by:

1. Collapsing duplicate rows
2. Reindexing to a continuous hourly timeline

**1. Collapsing Duplicate Rows**

The bike-sharing system should record **exactly one entry per hour**. If we find multiple rows for the same timestamp, something has gone wrong. These duplicates can arise for several reasons:

- **Data collection partitioning**: Different sensors or stations reporting separately for the same hour
- **System processing delays**: Multiple data collection cycles within the same hour  
- **Data pipeline issues**: ETL processes creating duplicate records

In all of these situations, the extra rows are not separate time periods but **fragments of the same underlying hour**. This is why we adopt the working assumption that duplicates represent **partial data for a single hour**.

Instead of discarding these rows, we will **combine them into one consolidated record**. This approach is practical for our project, but remember that in real consulting work you should always **validate the duplication patterns before applying aggregation**, to confirm that duplicates really are fragments and not a sign of a different data issue.

Since we assume that duplicates are **partial fragments of the same hour**, we can combine them into a single record using the following aggregation policy:

- For **targets** (`count`, `casual`, `registered`): use the **SUM**, because bike rentals accumulate across fragments. Two partial records for the same hour simply add up to the true total demand.
- For **numeric predictors** (e.g., weather variables): take the **MEAN**, since these are measured as continuous conditions. Averaging across fragments best represents the overall hourly state.
- For **categorical variables** (`holiday`, `weather`, `season`): take the **FIRST** value, because these codes should remain stable within an hour. A duplicated categorical code does not add new information.

In [ ]:
import numpy as np

# Identify duplicated timestamps
rows_per_ts = df.groupby("datetime").size().rename("n_rows_per_ts")
duplicated_ts = rows_per_ts[rows_per_ts > 1].index
n_dup_timestamps = len(duplicated_ts)
n_dup_rows = int((rows_per_ts[rows_per_ts > 1] - 1).sum())

print(f"Duplicated timestamps: {n_dup_timestamps} | Extra duplicate rows to collapse: {n_dup_rows}")

# Build aggregation policy
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
agg_map = {c: "mean" for c in numeric_cols}

for c in ["count", "casual", "registered"]:
    if c in agg_map:
        agg_map[c] = "sum"

for c in ["holiday", "weather", "season"]:
    if c in df.columns:
        agg_map[c] = "first"

# Aggregate to one row per hour
df = (
    df.groupby("datetime", as_index=False)
      .agg(agg_map)
      .sort_values("datetime")
      .reset_index(drop=True)
)

# Add a flag for hours that were collapsed from duplicates
df["flag_collapsed_from_duplicates"] = df["datetime"].isin(duplicated_ts)

print("Collapse complete → policy: SUM targets, MEAN numeric predictors, FIRST categoricals.")
print("Hours affected:", int(df["flag_collapsed_from_duplicates"].sum()))

We collapsed **24 duplicated hours**, removing 24 extra rows from the dataset. The aggregation policy ensures that:

- Demand counts remain correct (no double-counting).
- Weather predictors reflect average conditions.
- Categorical codes remain stable.

The flag `flag_collapsed_from_duplicates` marks these hours so we can always trace which rows were affected. For transparency, this is important: if a client later asks why certain hours look unusual, we can point to the duplication issue.

**2. Reindexing to a Continuous Hourly Timeline**

Finally, we enforce a **continuous hourly index**. Right now, the dataset simply skips missing hours — they aren’t represented at all. This makes gaps invisible and impossible to handle systematically.

By reindexing:

- We insert a row for **every missing hour**.
- Those rows will contain `NaN`s for predictors and/or target.
- We add a flag to mark which rows were **inserted**.

Next, we'll enforce a continuous hourly timeline by inserting rows for missing hours. This makes all gaps explicit and manageable.

In [ ]:
# Build the full hourly index
time_min = df["datetime"].min()
time_max = df["datetime"].max()
full_hours = pd.date_range(time_min, time_max, freq="h")

# Keep original set of hours
original_hours = pd.Index(df["datetime"])

# Reindex and flag
df = df.set_index("datetime").reindex(full_hours)
df.index.name = "datetime"
df["flag_missing_timestamp"] = ~df.index.isin(original_hours)

# Audit
inserted_hours = int(df["flag_missing_timestamp"].sum())
total_hours = len(df)
present_hours = total_hours - inserted_hours

print("=== Reindex audit ===")
print("time_min:", time_min)
print("time_max:", time_max)
print("total_hours_after_reindex:", total_hours)
print("present_hours_from_source:", present_hours)
print("inserted_missing_hours:", inserted_hours)

After reindexing, the dataset now has **17,256 rows** — one for each expected hour. Of these, **6,394 rows were inserted** to represent missing hours. These rows currently contain `NaN`s, which is exactly what we want: the gaps are now explicit and can be handled in the cleaning workflow.

From here:

- For **predictors**, missing values can be imputed using interpolation or seasonal medians.
- For the **target (`count`)**, rows with missing demand must be dropped before model training.
- The flag `flag_missing_timestamp` allows us to communicate clearly to clients how much of the dataset is reconstructed rather than observed.

With these two steps, we’ve established a **reliable timeline**. The dataset now has exactly one row per hour, duplicates resolved, and missing periods made explicit. This creates the solid foundation we need before applying the unified cleaning workflow to outliers and missing data.

### 4.3. Applying the Workflow to Outliers

Earlier we learned to detect outliers; now we'll apply the unified workflow to treat them systematically. The key shift is from detection to decision-making.

Let's work through the three-step workflow systematically with our detected outliers:

**Step 1: Is this an event or an error?**

First, we examine each outlier to determine its nature. This step requires both data analysis and business context.

*Physically Impossible Values*

Some outliers are easy to classify because they are **physically impossible**:

- Negative rentals (`count < 0`) — cannot exist in any transportation system
- Humidity above 100% — violates physical laws
- Temperatures above 100°C in Washington, D.C. — impossible for the local climate

In [ ]:
import numpy as np

# Identify impossible values across different variables
print("=== Step 1: Identifying impossible values (clear errors) ===")

# Check negative rentals
negative_rentals = (df["count"] < 0).sum()
print(f"Negative rental counts: {negative_rentals}")

# Check invalid humidity
invalid_humidity = ((df["humidity"] < 0) | (df["humidity"] > 100)).sum()
print(f"Humidity outside 0-100%: {invalid_humidity}")

# Check extreme temperatures (conservative cutoff for Washington climate)
extreme_temps = (df["temp"] > 60).sum()
print(f"Temperatures above 60°C: {extreme_temps}")

**Decision:** These are **always errors** — they cannot represent real-world events. Move to Step 2.

*Extreme-but-Possible Values*

Other outliers are statistically extreme but could potentially be real events:

In [ ]:
# Check for extreme daily rental patterns using Z-scores
daily_rentals = (
    df["count"]
    .groupby(df.index.date)
    .sum()
    .rename("daily_count")
    .reset_index()
)

mean = daily_rentals["daily_count"].mean()
std = daily_rentals["daily_count"].std()
daily_rentals["z_score"] = (daily_rentals["daily_count"] - mean) / std

# Flag potential outliers (|Z| > 3)
extreme_days = daily_rentals[daily_rentals["z_score"].abs() > 3]

print(f"\n=== Extreme daily rental patterns ===")
print(f"Days with |Z-score| > 3: {len(extreme_days)}")
extreme_days.head()

**Decision:** These require investigation. Could be:

- **Events**: Festival days, unusual weather patterns, special promotions
- **Errors**: Duplicated logs, sensor malfunctions, data processing issues

For this example, we'll assume days with extreme rentals could be legitimate events (holidays, festivals) and should be kept but flagged for transparency.

**Step 2: If error, can I fix it with a rule?**

For the values we've classified as errors, we apply rule-based fixes where possible:

In [ ]:
print("=== Step 2: Applying rule-based fixes to errors ===")

# Fix impossible values using business rules

# Negative rentals: Set to NaN (cannot be fixed with a rule)
df["flag_negative_count"] = df["count"] < 0
df.loc[df["flag_negative_count"], "count"] = np.nan
print(f"Negative rentals set to NaN: {df['flag_negative_count'].sum()}")

# Humidity outside valid range: Set to NaN (cannot determine correct value)
df["flag_humidity_invalid"] = (df["humidity"] < 0) | (df["humidity"] > 100)
df.loc[df["flag_humidity_invalid"], "humidity"] = np.nan
print(f"Invalid humidity values set to NaN: {df['flag_humidity_invalid'].sum()}")

# Extreme temperatures: Set to NaN (cannot determine correct value)
df["flag_temp_invalid"] = df["temp"] > 60
df.loc[df["flag_temp_invalid"], "temp"] = np.nan
print(f"Extreme temperatures set to NaN: {df['flag_temp_invalid'].sum()}")

**Decision:** These impossible values cannot be fixed with business rules (we don't know what the correct values should be), so they move to Step 3.

**Step 3: If cannot fix, should I impute or drop?**

For values that cannot be rule-based fixed, we decide between imputation and dropping based on variable type:

In [ ]:
print("\n=== Step 3: Impute or drop unfixable errors ===")

# For PREDICTORS (weather variables): These can be imputed
print("Weather predictors with NaN values after error correction:")
weather_vars = ["temp", "humidity", "windspeed"]
for var in weather_vars:
    if var in df.columns:
        nan_count = df[var].isna().sum()
        print(f"  {var}: {nan_count} NaN values → will be imputed later")

# For TARGET (count): Never impute for modeling - these rows will be dropped
target_nan = df["count"].isna().sum()
print(f"\nTarget variable 'count': {target_nan} NaN values → rows will be dropped before modeling")

# Flag extreme events that we're keeping
df["flag_extreme_but_kept"] = False
if len(extreme_days) > 0:
    extreme_dates = extreme_days["datetime"].dt.date
    df["flag_extreme_but_kept"] = df.index.date.isin(extreme_dates)
    print(f"Extreme-but-possible days flagged for transparency: {df['flag_extreme_but_kept'].sum()}")

**Summary of workflow application:**

1. **Step 1 (Event or Error):**
   - Impossible values → Classified as errors
   - Extreme-but-possible values → Classified as potential events, kept with flags

2. **Step 2 (Fix with rule):**
   - Impossible values → No business rule available, set to NaN and flagged

3. **Step 3 (Impute or drop):**
   - Predictor variables with NaN → Will be imputed in preprocessing
   - Target variable with NaN → Rows will be dropped before modeling
   - Extreme events → Kept with transparency flags

Following the unified workflow ensures that every outlier treatment decision is systematic, auditable, and defensible. Impossible values were identified as clear errors and set to missing, extreme-but-possible values were preserved with transparency flags, and the treatment strategy differs appropriately between predictors (impute) and targets (drop). Every intervention is flagged, ensuring complete traceability for client communications.

### 4.4. Applying the Workflow to Missing Data

Earlier we systematically detected missing data patterns; now we'll apply the unified workflow to treat them strategically. Missing data presents different challenges than outliers because absence itself can be meaningful information.

The key distinction is understanding **why** data is missing: system failures during storms tell a different story than random sensor glitches, and each requires different treatment strategies.

Let's work through the three-step workflow systematically with our detected missing data patterns:

**Step 1: Is this an event or an error?**

Missing data can result from legitimate operational events or system errors. Understanding the cause determines our treatment approach.

*Systematic Operational Gaps*

First, let's examine the temporal clustering we identified earlier — missing data concentrated in July periods:

In [ ]:
# Analyze missing data patterns by year-month
print("=== Step 1: Analyzing missingness patterns ===")

# Focus on the target variable and key predictors
key_vars = ["count", "temp", "humidity", "windspeed", "holiday"]
missing_analysis = {}

for var in key_vars:
    if var in df.columns:
        # Monthly missingness pattern
        monthly_missing = (
            df.groupby(df.index.to_period('M'))[var]
            .apply(lambda x: x.isna().sum())
            .rename(f"{var}_missing")
        )
        missing_analysis[var] = monthly_missing

# Display July patterns specifically
july_months = [period for period in monthly_missing.index if period.month == 7]
print(f"\nMissing data in July periods:")
for month in july_months:
    print(f"{month}:")
    for var in key_vars:
        if var in missing_analysis:
            count = missing_analysis[var].get(month, 0)
            print(f"  {var}: {count} missing hours")

**Decision:** The July clustering suggests **systematic operational events** — likely planned maintenance windows or sensor replacement cycles. This is normal system operation, not an error.

*Weather Station Outages*

Next, examine simultaneous missingness across weather variables:

In [ ]:
# Check for simultaneous weather variable failures
weather_vars = ["temp", "atemp", "humidity", "windspeed"]
available_weather_vars = [var for var in weather_vars if var in df.columns]

# Count rows where multiple weather variables are missing together
weather_missing_counts = df[available_weather_vars].isna().sum(axis=1)
simultaneous_failures = (weather_missing_counts >= 2).sum()

print(f"\nSimultaneous weather failures:")
print(f"Hours with 2+ weather variables missing: {simultaneous_failures}")

# Sample some simultaneous failure periods
simultaneous_missing_rows = df[weather_missing_counts >= 2].head()
print(f"\nExample simultaneous failures:")
for idx, row in simultaneous_missing_rows.iterrows():
    missing_vars = [var for var in available_weather_vars if pd.isna(row[var])]
    print(f"  {idx}: {missing_vars}")

**Decision:** Simultaneous weather failures indicate **weather station outages** — operational events, not errors. These represent real system limitations during specific periods.

*Holiday Information Gaps*

In [ ]:
# Analyze holiday missingness pattern
if "holiday" in df.columns:
    holiday_missing = df["holiday"].isna().sum()
    total_rows = len(df)
    holiday_missing_pct = (holiday_missing / total_rows) * 100
    
    print(f"\nHoliday information gaps:")
    print(f"Missing holiday labels: {holiday_missing} ({holiday_missing_pct:.1f}%)")
    
    # Check if missing holidays cluster around known holiday periods
    holiday_missing_months = (
        df[df["holiday"].isna()]
        .groupby(df.index.to_period('M'))
        .size()
        .sort_values(ascending=False)
    )
    print(f"Holiday missingness by month (top 5):")
    print(holiday_missing_months.head())

**Decision:** Missing holiday labels are likely **data processing gaps** rather than operational events. These can potentially be filled using external holiday calendars.

**Step 2: If error, can I fix it with a rule?**

Since we classified July clustering and weather station outages as **operational events** (not errors) in Step 1, only the holiday information gaps require rule-based fixing:

In [ ]:
print("\n=== Step 2: Applying rule-based fixes to errors ===")

# Fix holiday information using business rules
if "holiday" in df.columns:
    # Create a simple holiday calendar for major US federal holidays
    # In practice, you'd use a comprehensive holiday library
    known_holidays = {
        '2011-01-01': 1, '2011-07-04': 1, '2011-12-25': 1,
        '2012-01-01': 1, '2012-07-04': 1, '2012-12-25': 1,
        # Add more holidays as needed
    }
    
    # Flag original missing holidays
    df["flag_holiday_originally_missing"] = df["holiday"].isna()
    
    # Fill known holidays
    holiday_fixes = 0
    for date_str, holiday_val in known_holidays.items():
        date = pd.to_datetime(date_str)
        if date in df.index and pd.isna(df.loc[date, "holiday"]):
            df.loc[date, "holiday"] = holiday_val
            holiday_fixes += 1
    
    print(f"Holiday values filled using calendar rules: {holiday_fixes}")
    
    # Remaining missing holidays default to 0 (non-holiday)
    remaining_missing = df["holiday"].isna().sum()
    df["holiday"] = df["holiday"].fillna(0)
    print(f"Remaining missing holidays set to 0 (non-holiday): {remaining_missing}")

print("\nNote: Weather variables and July patterns are operational events, not errors")
print("→ They proceed directly to Step 3 for imputation strategy")

**Decision:** Holiday information gaps were successfully addressed using external calendar rules. Weather station outages and maintenance periods are legitimate operational events that require imputation rather than rule-based fixes.

**Step 3: If cannot fix, should I impute or drop?**

For unfixable missing values, we decide based on variable type and modeling requirements:

In [ ]:
print("\n=== Step 3: Impute or drop strategy ===")

# Strategy for PREDICTORS (weather and other features)
print("Weather predictors with missing values after error correction:")
for var in available_weather_vars:
    if var in df.columns:
        missing_count = df[var].isna().sum()
        if missing_count > 0:
            # Create flag for transparency
            flag_name = f"flag_{var}_missing"
            df[flag_name] = df[var].isna()
            print(f"  {var}: {missing_count} missing values → will be imputed later")

# Strategy for TARGET variable
print(f"\nMissing target variable - DROPPING strategy:")
target_missing = df["count"].isna().sum()
df["flag_target_missing"] = df["count"].isna()

print(f"  'count': {target_missing} rows with missing target → will be dropped before modeling")
print(f"  Reason: Never impute target variables for supervised learning")

# Strategy for operational gaps (preserve context)
print(f"\nOperational gaps - PRESERVE WITH CONTEXT:")
df["flag_operational_gap"] = df["flag_missing_timestamp"] | df["flag_collapsed_from_duplicates"]
operational_gaps = df["flag_operational_gap"].sum()
print(f"  {operational_gaps} hours affected by operational events → preserved with context flags")

**Summary of workflow application:**

1. **Step 1 (Event or Error):**
   - July clustering → Operational maintenance events, preserved with context
   - Weather station outages → Operational limitations, imputation needed
   - Holiday gaps → Data processing errors, fixable with rules

2. **Step 2 (Fix with rule):**
   - Holiday information → Partially fixed using calendar rules

3. **Step 3 (Impute or drop):**
   - Predictor variables → Flagged for later imputation during preprocessing
   - Target variable → Rows with missing targets dropped (never impute)
   - Operational gaps → Preserved with context for client transparency

The missing data workflow ensures that every gap is handled appropriately based on its cause and variable type. Operational events are preserved with context, fixable gaps are corrected systematically, and remaining missingness is flagged for later preprocessing (predictors) or dropping (targets) with complete transparency. This approach maintains data integrity while deferring sophisticated imputation to the preprocessing phase where it belongs.

### 4.5. Professional Data Cleaning Checklist

Here's your consultant-ready workflow for any data cleaning project:

**Phase 1: Foundation**
- [ ] Standardize timeline (collapse duplicates, enforce continuity)
- [ ] Add flags for all structural changes

**Phase 2: The Three-Step Decision Process**
For every suspicious value:
- [ ] **Step 1:** Event or error? (Context check)
- [ ] **Step 2:** If error, can I fix with a rule? (Apply fix + flag)
- [ ] **Step 3:** If unfixable, impute or drop? (Predictors vs. targets)

**Phase 3: Documentation**
- [ ] Flag every intervention with clear labels
- [ ] Document aggregation policies and business logic
- [ ] Quantify impact ("15% of weather data imputed")
- [ ] Prepare client communication on data limitations

**Quality Gates:**
- ✅ No impossible values remain
- ✅ Timeline is continuous and complete
- ✅ Every change is flagged and traceable
- ✅ Client can understand what was done and why

This checklist ensures your cleaning is not just thorough, but **defendable to clients and auditable by colleagues**.

---

## Summary and Transition to Feature Engineering Implementation

You've mastered essential data quality assessment and cleaning techniques: systematic missing data analysis, outlier detection, and validation workflows. These skills transform messy real-world datasets into reliable, analytical-grade information.

Your expertise in data completeness assessment, systematic cleaning procedures, and quality validation creates the trustworthy foundation needed for all advanced analysis. This technical rigor combined with business judgment enables you to work confidently with complex transportation datasets while maintaining analytical integrity.

In our next lecture, we'll build on this clean data foundation by learning advanced feature engineering and preprocessing techniques that prepare your data for machine learning models. You'll see how quality data translates into effective model-ready features for predictive analytics.